In [5]:
# Setup Ring of Differential Polynomials for two functions u and v
# u_i = d^i u / dx^i, v_i = d^i v / dx^i
R = PolynomialRing(QQ, ['u_%d'%i for i in range(20)] + ['v_%d'%i for i in range(20)])
gens = R.gens()
u = gens[:20]
v = gens[20:]

def deriv(f):
    """Total derivative D_x for both u and v"""
    res = 0
    for i in range(19):
        res += f.derivative(u[i]) * u[i+1]
        res += f.derivative(v[i]) * v[i+1]
    return res

def deriv_n(f, n):
    for _ in range(n): f = deriv(f)
    return f

def pseudo_mul(A, B, max_prec=10):
    res = {}
    for powA, coeffA in A.items():
        for powB, coeffB in B.items():
            for j in range(max_prec):
                new_pow = powA + powB - j
                if powA >= 0:
                    b_coeff = binomial(powA, j)
                else:
                    b_coeff = (-1)**j * binomial(-powA + j - 1, j)
                term_coeff = coeffA * deriv_n(coeffB, j) * b_coeff
                res[new_pow] = res.get(new_pow, 0) + term_coeff
    return {p: c for p, c in res.items() if c != 0 and p > -max_prec}

def compute_L_one_third(n_terms=10):
    """Solves Q^3 = d^3 + u*d + v for Q = d + b0 + b1*d^-1..."""
    Q = {1: R(1)} # d^1
    for k in range(0, n_terms):
        # Calculate Q^3 with current coefficients
        Q2 = pseudo_mul(Q, Q, max_prec=n_terms+2)
        Q3 = pseudo_mul(Q2, Q, max_prec=n_terms+2)
        
        target = {3: R(1), 1: u[0], 0: v[0]}
        # The coefficient of d^(3-k-1) allows us to solve for the next b
        # For Q^3, the lead term for the unknown b is 3 * d^2 * b
        diff = target.get(2-k, 0) - Q3.get(2-k, 0)
        Q[-k] = diff / 3
    return Q

def get_boussinesq_flow(m):
    L_third = compute_L_one_third(m + 4)
    Lm_third = {0: R(1)}
    for _ in range(m):
        Lm_third = pseudo_mul(Lm_third, L_third, max_prec=m+2)
    
    Lm_plus = {p: c for p, c in Lm_third.items() if p >= 0}
    L = {3: R(1), 1: u[0], 0: v[0]}
    
    # [Lm_plus, L]
    comm = pseudo_mul(Lm_plus, L)
    m_comm = pseudo_mul(L, Lm_plus)
    
    res = {}
    for p in set(comm.keys()) | set(m_comm.keys()):
        val = comm.get(p, 0) - m_comm.get(p, 0)
        if val != 0: res[p] = val
    return res

# --- Execute for m=1 and m=2 ---
for m in [1, 2, 3, 4]:
    flow = get_boussinesq_flow(m)
    print(f"\n--- Boussinesq Hierarchy m={m} ---")
    # In dL/dt = [P, L], the coeff of d^1 is u_t, coeff of d^0 is v_t
    print(f"u_t = {flow.get(1, 0)}")
    print(f"v_t = {flow.get(0, 0)}")


--- Boussinesq Hierarchy m=1 ---
u_t = u_1
v_t = v_1

--- Boussinesq Hierarchy m=2 ---
u_t = -u_2 + 2*v_1
v_t = -2/3*u_0*u_1 - 2/3*u_3 + v_2

--- Boussinesq Hierarchy m=3 ---
u_t = 0
v_t = 0

--- Boussinesq Hierarchy m=4 ---
u_t = -2/3*u_1^2 - 2/3*u_0*u_2 + 4/3*u_1*v_0 + 4/3*u_0*v_1 - 1/3*u_4 + 2/3*v_3
v_t = -4/9*u_0^2*u_1 - 4/3*u_1*u_2 - 2/3*u_0*u_3 + 2/3*u_1*v_1 + 4/3*v_0*v_1 + 2/3*u_0*v_2 - 2/9*u_5 + 1/3*v_4
